In [1]:
from pyspark.sql import SparkSession, functions as F

In [3]:
spark = (SparkSession.builder
         .appName("Hive source")
         .master("yarn")
         .enableHiveSupport()
         .getOrCreate()
         
        )

2023-02-19 12:08:25,465 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-02-19 12:08:32,630 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [5]:
df = (spark.read.format("csv")
      .option("header", True)
      .option("inferSchema", True)
      .option("sep", ",")
      .load("hdfs://localhost:9000/user/train/datasets/Churn_Modelling.csv"))

In [6]:
df.limit(2).toPandas()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [8]:
df2 = df.filter("CreditScore > 600")

In [11]:
spark.sql("use test1;").show()

++
||
++
++



2023-02-19 12:14:10,826 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


In [14]:
spark.sql("show tables;").show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
|   test1|         advertising|      false|
|   test1|advertising_sales...|      false|
|   test1|              adv_lk|      false|
|   test1|       credit_gt_600|      false|
|   test1|          hotels_orc|      false|
|   test1|      hotels_parquet|      false|
|   test1|     hotels_parquet2|      false|
|   test1|         hotels_text|      false|
|   test1|             mytable|      false|
|   test1|sales_part_countr...|      false|
|   test1|sales_partitioned...|      false|
+--------+--------------------+-----------+



In [13]:
df2.write.format("orc") \
.mode("overwrite") \
.saveAsTable("test1.credit_gt_600")

2023-02-19 12:15:28,309 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
2023-02-19 12:15:28,508 WARN conf.HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
2023-02-19 12:15:28,508 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2023-02-19 12:15:28,510 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist


# Read from Hive

In [15]:
df_credit_gt_600 = spark.sql("select * from test1.credit_gt_600")

In [16]:
df_credit_gt_600.count()

6934